In [3]:
import numpy as np

## Task 1
Implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the following substitution function:
    $$f : y_j(j): 2v_i(j) \mod  p, \ j$$

In [4]:
def subkey_generator(key):

    k_1 = [key[0],key[2],key[4],key[6]] 
    k_2 = [key[0],key[1],key[2],key[6]] 
    k_3 = [key[0],key[3],key[4],key[6]] 
    k_4 = [key[0],key[3],key[5],key[6]] 
    k_5 = [key[0],key[2],key[5],key[6]] 
    k_6 = [key[2],key[4],key[4],key[6]] 

    return [k_1, k_2, k_3, k_4, k_5, k_6]

def subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter)

    return (w + k)%p



In [5]:
 
def timesTwo(v,p):

    return np.multiply(v,2)%p

def transposition(y):

    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])

def linear(z, p):

    z_matrix = z.reshape(2,4)

    param_matrix = np.matrix([[2,5],[1,7]])

    w_matrix = np.dot(param_matrix,z_matrix)%p

    w_array = np.asarray(w_matrix).flatten()

    return w_array
        




In [6]:
def encryption(u,k,p,round,substitution,transposition,subkey_sum,linear):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)         
        #print("post subkey_sum",v)
        y = substitution(v,p)
        #print("post timesTwo",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [7]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p=11
r = 5

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

print(x)


[4 0 0 9 7 0 0 3]


## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field $\mathbb{F} = GF(p)$.

In [8]:
def inv_timesTwo(v,p):
    mul_inv = pow(2,-1,p)
    return np.multiply(v,mul_inv)%p


def inv_transposition(y):
    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])


def inv_subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w - k)%p

#Calcolo dell'inversa della matrice modulo p
def inv_matrix(m, p):
    # Calcolo del determinante della matrice
    det = int(np.round(np.linalg.det(m)))  # Determinante della matrice
    det_inv = pow(det, -1, p)  # Inverso del determinante modulo p

    # Calcolo della matrice aggiunta (trasposta della matrice dei cofattori)
    cofactors = np.zeros_like(m, dtype=int)
    n = m.shape[0]
    for i in range(n):
        for j in range(n):
            minor = np.delete(np.delete(m, i, axis=0), j, axis=1)  # Minore
            cofactor = ((-1) ** (i + j)) * int(np.round(np.linalg.det(minor)))
            cofactors[i, j] = cofactor

    adj_matrix = cofactors.T  # Trasposta della matrice dei cofattori

    # Calcolo dell'inversa modulo p
    m_inv = (det_inv * adj_matrix) % p

    return m_inv

def inv_linear(z, p):
    z_matrix = z.reshape(2, 4)

    param_matrix = np.array([[2, 5], [1, 7]])

    #Calcolo dell'inversa della matrice modulo p
    param_matrix_inv = inv_matrix(param_matrix, p)

    w_matrix = np.dot(param_matrix_inv, z_matrix) % p

    w_array = np.asarray(w_matrix, dtype=int).flatten()

    return w_array


def decryption(x,k,p,round):
    k_list = subkey_generator(k)

    for i in range(round,-1,-1):
        print("round : ", i)

        z = inv_subkey_sum(k_list[i],x,p)            #v = u - k
        print("post subkey_sum",z)
        
        if i == 0:
            x = z
            break
        elif i != round:
            w = inv_linear(z, p)
            print("post linear", w)
        else:
            w = z

        y = inv_transposition(w)
        print("post transpo",y)

        x = inv_timesTwo(y,p)
        print("post timesTwo",x)
        
    return x

Test for inv_linear

In [9]:
a = linear(np.array([1,2,3,5,4,6,4,8]), p)  #change the array to test
print(a)
b = inv_linear(a, p)
print(b)

[0 1 4 6 7 0 9 6]
[1 2 3 5 4 6 4 8]


In [10]:
u = decryption(x,k,p,r)
print(u)

round :  5
post subkey_sum [4 0 0 9 7 0 0 3]
post transpo [4 0 0 9 3 0 0 7]
post timesTwo [ 2  0  0 10  7  0  0  9]
round :  4
post subkey_sum [ 1  0  0 10  6  0  0  9]
post linear [6 0 0 4 0 0 0 7]
post transpo [6 0 0 4 7 0 0 0]
post timesTwo [3 0 0 2 9 0 0 0]
round :  3
post subkey_sum [2 0 0 2 8 0 0 0]
post linear [2 0 0 4 4 0 0 1]
post transpo [2 0 0 4 1 0 0 4]
post timesTwo [1 0 0 2 6 0 0 2]
round :  2
post subkey_sum [0 0 0 2 5 0 0 2]
post linear [ 7  0  0  9  6  0  0 10]
post transpo [ 7  0  0  9 10  0  0  6]
post timesTwo [ 9  0  0 10  5  0  0  3]
round :  1
post subkey_sum [ 8  0  0 10  4  0  0  3]
post linear [4 0 0 0 0 0 0 2]
post transpo [4 0 0 0 2 0 0 0]
post timesTwo [2 0 0 0 1 0 0 0]
round :  0
post subkey_sum [1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]


Other tests for decryption

In [11]:
a = np.array([1,8,3,6,8,6,9,2]) #change plaintext to test
b = np.array([1,0,4,0,3,0,2,0]) #change key to test
c = encryption(a,b,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

print(c)
d = decryption(c,b,p,r)
print(d)

[ 7  9  1  5 10  0 10  1]
round :  5
post subkey_sum [ 3  6  9  3  6  8  7 10]
post transpo [ 3  6  9  3 10  7  8  6]
post timesTwo [ 7  3 10  7  5  9  4  3]
round :  4
post subkey_sum [ 6 10 10  5  4  5  4  1]
post linear [ 0  5  8  7 10  0  1  7]
post transpo [ 0  5  8  7  7  1  0 10]
post timesTwo [0 8 4 9 9 6 0 5]
round :  3
post subkey_sum [10  8  4  7  8  6  0  3]
post linear [7 9 8 5 8 9 2 6]
post transpo [7 9 8 5 6 2 9 8]
post timesTwo [ 9 10  4  8  3  1 10  4]
round :  2
post subkey_sum [ 8 10  1  6  2  1  7  2]
post linear [10  6  3  6  2  4 10  1]
post transpo [10  6  3  6  1 10  4  2]
post timesTwo [5 3 7 3 6 5 2 1]
round :  1
post subkey_sum [ 4  3  3  1  5  5  9 10]
post linear [4 2 1 5 8 2 9 7]
post transpo [4 2 1 5 7 9 2 8]
post timesTwo [ 2  1  6  8  9 10  1  4]
round :  0
post subkey_sum [1 8 3 6 8 6 9 2]
[1 8 3 6 8 6 9 2]


## Task 3
Identify the overall linear relationship for this simplified AES-like cipher, that is find the
matrices $A ∈ F^{(ℓ_x × ℓ_k)}$ and $B ∈ F^{(ℓ_x × ℓ_u)}$ such that
$$x = E(k, u) = Ak + Bu \mod p$$
with all operations in the field $\mathbb{F} = GF(p)$.

In [12]:
def linear_relationship():  #pass the encryption function as a parameter
    e_a = np.eye(8, dtype=int)
    k_a = np.zeros(8, dtype=int)
    A = np.empty((8,0), dtype=int)

    k_b = np.eye(8, dtype=int)
    e_b = np.zeros(8, dtype=int)
    B = np.empty((8,0), dtype=int)

    for i in range(8):

        encrypted_value = encryption(k_a, np.asarray(e_a[i]).flatten(), p, r,
                                    substitution=timesTwo,
                                    transposition=transposition,
                                    subkey_sum=subkey_sum,
                                    linear=linear)
        
        A = np.hstack((A, encrypted_value.reshape(-1, 1)))

    #print(A)

    for i in range(8):

        encrypted_value = encryption(np.asarray(k_b[i]).flatten(),e_b,p,r,
                                    substitution=timesTwo,
                                    transposition=transposition,
                                    subkey_sum=subkey_sum,
                                    linear=linear)
        B = np.hstack((B, encrypted_value.reshape(-1, 1)))


    #print(B)

    return A, B

In [13]:
x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

A, B = linear_relationship()
print(x == (np.dot(A,k) + np.dot(B,u))%p)

[ True  True  True  True  True  True  True  True]


In [14]:
a = np.array([1,8,4,6,8,6,5,2]) #change plaintext to test
b = np.array([1,6,5,1,3,6,2,0]) #change key to test

x = encryption(a,b,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

A, B = linear_relationship()

print(x == (np.dot(A,b) + np.dot(B,a))%p)

[ True  True  True  True  True  True  True  True]


## Task 4
From a known plaintext/ciphertext pair $(u, x)$, implement a linear cryptanalysis KPA against this cipher by computing 
$$k = A^{−1}(x − Bu) \mod p$$ 
with all operations in the field $\mathbb{F} = GF(p)$

In [15]:
import numpy as np
import scipy as sp
from itertools import product

def null_space_mod_p(A, p):
    #Compute the null space basis
    null_space_basis = sp.linalg.null_space(A)

    #If the null space is empty, return an empty list
    if null_space_basis.size == 0:
        return []

    #Convert the null space basis to integers and reduce modulo p
    null_space_basis_mod_p = [(np.round(vec).astype(int) % p) for vec in null_space_basis.T]

    #Generate all linear combinations of the basis vectors modulo p
    num_basis_vectors = len(null_space_basis_mod_p)
    coeff_range = range(p)  # Coefficients from 0 to p-1
    all_null_space_elements = []

    for coeffs in product(coeff_range, repeat=num_basis_vectors):
        element = sum(c * v for c, v in zip(coeffs, null_space_basis_mod_p)) % p
        all_null_space_elements.append(element)

    return all_null_space_elements


In [ ]:
def KPA(A, B, p, u, x):
    Bu = np.dot(B,u)%p
    b = (-Bu + x) % p

    x_p, _, _, _ = sp.linalg.lstsq(A, b)
    x_p = np.round(x_p).astype(int) % p
    print(x_p)
    
    # Compute the null space of A modulo p
    all_null_space_elements_A = null_space_mod_p(A, p)

    for vec in all_null_space_elements_A:
        h = (x_p + vec)%p
        # Check if h is a valid key
        result = (np.dot(A,h) + np.dot(B,u) - x) % p
        print(result)
        if np.all(result == 0):
            print(f"Found key: {h}")
            break

In [58]:
A, B = linear_relationship()
x = encryption(u,k,p,r,substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)
p = 11
null_space_A = null_space_mod_p(A, p)

print(len(null_space_A))
print("Checking vectors in the null space of A modulo", p, ":")
for vec in null_space_A:
    # Verify that A * vec ≡ 0 (mod p)
    check = (np.dot(A, vec) % p)
    if np.all(check == 0):  # Check if all elements are 0 modulo p
        print(f"Vector {vec} is in the null space of A modulo {p}.")
    else:
        print(f"Vector {vec} is NOT in the null space of A modulo {p}.")

print(A)


11
Checking vectors in the null space of A modulo 11 :
Vector [0 0 0 0 0 0 0 0] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 1] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 2] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 3] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 4] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 5] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 6] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 7] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 8] is in the null space of A modulo 11.
Vector [0 0 0 0 0 0 0 9] is in the null space of A modulo 11.
Vector [ 0  0  0  0  0  0  0 10] is in the null space of A modulo 11.
[[ 9  0  1  0  0  0  6  0]
 [ 0  8  6  1  3  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  0  0  0 10  0]
 [ 2  0  1  0  0  0  3  0]
 [ 0  1  8  3 10  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  0  0  0  3  0]]


In [59]:
A, B = linear_relationship()
x = encryption(u,k,p,r,substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)
p = 11

KPA(A, B, p, u, x)

[1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
Found key: [1 0 0 0 0 0 0 0]


In [60]:
A, B = linear_relationship()
a = np.array([5,9,9,9,2,5,9,1]) #change plaintext
b = np.array([2,10,8,9,10,0,10,7]) #change key
c = encryption(a,b,p,r,substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear)

KPA(A,B,p,a,c)

(None, None)

## Task 5
implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the substitution function described by the following table
|$v_i(j)$|0|1|2|3|4|5|6|7|8|9|10|
|--------|-|-|-|-|-|-|-|-|-|-|--|
|$y_i(j)$|0|2|4|8|6|10|1|3|5|7|9|

where $j ∈ \{1, . . . , ℓ\}$

In [30]:
def custom_substitution_number(v):
    if(v == 0):
        return 0
    if(v == 1):
        return 2
    if(v == 2):
        return 4
    if(v == 3):
        return 8
    if(v == 4):
        return 6
    if(v == 5):
        return 10
    if(v == 6):
        return 1
    if(v == 7):
        return 3
    if(v == 8):
        return 5
    if(v == 9):
        return 7
    if(v == 10):
        return 9
    
def custom_substitution(v,p):

    for i in range(8):
        v[i] = custom_substitution_number(v[i])
    
    return v


def encryption_v2(u,k,p,round):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)
        #print("post subkey_sum",v)
        y = custom_substitution(v,p)
        #print("post custom substitution",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [31]:
print(encryption(u,k,p,r,
                 substitution=custom_substitution,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear
                 ))

[9 8 0 7 6 2 7 9]


## Task 6
Linear cryptanalysis of a “nearly linear” cipher

## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters: $\mathcal{K} = \mathbb{F}^{\ell_k}, \ell_k = 4$

ECC...

In [32]:
def subkey_generator_v2(key):

    k_1 = [key[0],key[1],key[2],key[3]]
    k_2 = [key[0],key[1],key[3],key[2]]
    k_3 = [key[1],key[2],key[3],key[0]]
    k_4 = [key[0],key[3],key[1],key[2]]
    k_5 = [key[2],key[3],key[0],key[1]]
    k_6 = [key[1],key[3],key[0],key[2]]

    return [k_1,k_2,k_3,k_4,k_5,k_6]

def mod_inv(v, p):
    
    x =[]

    for num in v:
        if(num != 0):
            x.append(2*pow(num, -1, p))
        else:
            x.append(0)


    return np.array(x) # Calcola l'inverso di x modulo p

def substitution(x):
    x= x.tolist()
    return mod_inv(x, p)
    

In [33]:
def encryption_v3(u,k,p,round):

    k_list = subkey_generator_v2(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)
        #print("post subkey_sum",v)
        y = substitution(v)
        #print("post custom substitution",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x

In [34]:
k=np.array([1,0,0,0])

p= 11

print(encryption_v3(u,k,p,r))

[5 0 3 2 5 2 1 1]
